In [7]:
from pyspark import SparkConf, SparkContext
import re

def load_names():
    names_map = {}
    with open("/home/jovyan/work/data/Marvel-names.txt", encoding='utf-8', errors='ignore') as f:
        for line in f:
            z = re.match('^(.*)\s\"(.*)\"$', line)
            if z:
                groups = z.groups()
                _id, _name = groups
                names_map[int(_id)] = _name
            else:
                print('ignore line', line)
    return names_map

conf = SparkConf().setMaster("local").setAppName("PopularSuperHeros")
sc = SparkContext.getOrCreate(conf = conf)

names_map = load_names()

lines = sc.textFile("file:///home/jovyan/work/data/Marvel-graph.txt")
def parse_line(line):
    ids = list(map(int, line.split()))
    origin = ids[0]
    for other_id in ids[1:]:
        yield (origin, other_id)
        
heroes_relations = lines.flatMap(parse_line)

occurences = heroes_relations.map(lambda relation: (relation[1], 1))

occurences_counts = occurences.reduceByKey(lambda x, y: x + y)

max_hero = occurences_counts.reduce(lambda hero1, hero2: hero1 if hero1[1]>hero2[1] else hero2)

print(names_map[max_hero[0]], max_hero[1])

CAPTAIN AMERICA 1933
